In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
%matplotlib inline

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# auto complete
%config Completer.use_jedi = False

In [14]:
# not used?
# o2Saturation_df = pd.read_csv('/kaggle/input/heart-attack-analysis-prediction-dataset/o2Saturation.csv')
# o2Saturation_df.head()

heart_df = pd.read_csv('./heart.csv')
heart_df.head()

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
0,63,1,3,145,233,1,0,150,0,2.3,0,0,1,1
1,37,1,2,130,250,0,1,187,0,3.5,0,0,2,1
2,41,0,1,130,204,0,0,172,0,1.4,2,0,2,1
3,56,1,1,120,236,0,1,178,0,0.8,2,0,2,1
4,57,0,0,120,354,0,1,163,1,0.6,2,0,2,1


In [15]:
heart_df.nunique()

age          41
sex           2
cp            4
trtbps       49
chol        152
fbs           2
restecg       3
thalachh     91
exng          2
oldpeak      40
slp           3
caa           5
thall         4
output        2
dtype: int64

In [5]:
len(heart_df) #303
heart_df.describe()
# 303 records for all cols => no missing values

# only numerical values (categorical features are already converted)

,age,sex,cp,trtbps,chol,fbs,restecg,thalachh,exng,oldpeak,slp,caa,thall,output
count,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000,303.000000
mean,54.366337,0.683168,0.966997,131.623762,246.264026,0.148515,0.528053,149.646865,0.326733,1.039604,1.399340,0.729373,2.313531,0.544554
std,9.082101,0.466011,1.032052,17.538143,51.830751,0.356198,0.525860,22.905161,0.469794,1.161075,0.616226,1.022606,0.612277,0.498835
min,29.000000,0.000000,0.000000,94.000000,126.000000,0.000000,0.000000,71.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,47.500000,0.000000,0.000000,120.000000,211.000000,0.000000,0.000000,133.500000,0.000000,0.000000,1.000000,0.000000,2.000000,0.000000
50%,55.000000,1.000000,1.000000,130.000000,240.000000,0.000000,1.000000,153.000000,0.000000,0.800000,1.000000,0.000000,2.000000,1.000000
75%,61.000000,1.000000,2.000000,140.000000,274.500000,0.000000,1.000000,166.000000,1.000000,1.600000,2.000000,1.000000,3.000000,1.000000
max,77.000000,1.000000,3.000000,200.000000,564.000000,1.000000,2.000000,202.000000,1.000000,6.200000,2.000000,4.000000,3.000000,1.000000


In [6]:
features = ['age', 'sex', 'cp', 'trtbps', 'chol', 'fbs', 'restecg', 'thalachh','exng', 'oldpeak', 'slp', 'caa', 'thall']
output = ['output']

X = heart_df[features]
y = heart_df[output]

In [28]:
# libraries
from sklearn.model_selection import train_test_split

X_train, X_valid, y_train, y_valid = train_test_split(
    X,
    y,
    train_size=0.8,
    test_size=0.2,
    random_state=0
)

In [29]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import xgboost as xgb

In [30]:
xg_class = xgb.XGBClassifier()

log1 = LogisticRegression(penalty='l2', solver='sag')
log2 = LogisticRegression(penalty='none', solver='sag')
log3 = LogisticRegression(penalty='l1', solver='liblinear')
log4 = LogisticRegression(penalty='l2', solver='liblinear')

svc1 = SVC(kernel='linear')
svc2 = SVC(kernel='poly')
svc3 = SVC(kernel='rbf')
svc4 = SVC(kernel='sigmoid')

sgd1 = SGDClassifier(loss='hinge')
sgd2 = SGDClassifier(loss='log')

tree1 = DecisionTreeClassifier(max_leaf_nodes=2)

models = [
    log1,
    log2,
    log3,
    log4,
    svc1,
    svc2,
    svc3,
    svc4,
    sgd1,
    sgd2,
    tree1,
    xg_class
]

In [34]:
from sklearn import metrics

results = pd.DataFrame(columns=['Model', 'Accuracy'])

for model in models:
    print(model)
    
    model.fit(X_train, y_train.values.ravel())
    
    preds = model.predict(X_valid)

    results.loc[len(results)] = [
        str(model),
        metrics.accuracy_score(preds, y_valid)
    ]
    
results.sort_values(by='Accuracy', ascending=False)

LogisticRegression(solver='sag')
LogisticRegression(penalty='none', solver='sag')
LogisticRegression(penalty='l1', solver='liblinear')
LogisticRegression(solver='liblinear')
SVC(kernel='linear')


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\linear_model\_sag.py:328: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn("The max_iter was reached which means "


SVC(kernel='poly')
SVC()
SVC(kernel='sigmoid')
SGDClassifier()
SGDClassifier(loss='log')
DecisionTreeClassifier(max_leaf_nodes=2)
XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
              importance_type='gain', interaction_constraints='',
              learning_rate=0.300000012, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=nan, monotone_constraints='()',
              n_estimators=100, n_jobs=12, num_parallel_tree=1, random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)
[16:01:34] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to res

C:\ProgramData\Anaconda3\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


,Model,Accuracy
2,"LogisticRegression(penalty='l1', solver='libli...",0.852459
3,LogisticRegression(solver='liblinear'),0.852459
4,SVC(kernel='linear'),0.819672
10,DecisionTreeClassifier(max_leaf_nodes=2),0.786885
11,"XGBClassifier(base_score=0.5, booster='gbtree'...",0.786885
0,LogisticRegression(solver='sag'),0.754098
1,"LogisticRegression(penalty='none', solver='sag')",0.754098
9,SGDClassifier(loss='log'),0.737705
5,SVC(kernel='poly'),0.721311
6,SVC(),0.688525
